### **Reminder:** 

### For each notebook, you should submit your notebook with all the cells executed, from start to finish, in a fresh run (i.e., first cell number should be [1], second [2], etc.). You can achieve this by selecting Kernel -> Restart and run all. At the end of the run, your notebook should have populated a database in the university server which you will have created exclusively for this coursework.

### **Any deviation from the submission instructions above will result in a mark of zero for the assessment or question part.**

# Part 1: NOTEBOOK: DATABASE CREATION AND QUERYING [30 marks]

In [1]:
import pymysql
import pandas as pd

In [2]:
connection = pymysql.connect(
    host='csmysql.cs.cf.ac.uk',
    user='c21051997',
    password='14Swissvalley!',
    database='c21051997_cmt655_coursework_db',
    charset='utf8mb4')

PART A: DATA MODELING [10 marks]

1) Python logic for reading in the data and handling missing values. Use of Pandas is recommended. Note: you can reuse code from Part 2 questions 1 and 3 for this question. [1 mark]

In [3]:
# your code here

# Read the CSV file containg the data, store into a "DataFrame"
df = pd.read_csv("art.csv")

# Replace any missing values in the DataFrame with "\\N"
df.fillna("\\N", inplace=True)

# Save the cleaned DataFrame into a new CSV file called cleaned_paintings, not including the index column
df.to_csv("cleaned_paintings.csv", index=False)

2) SQL code for creating tables. [4 marks]

In [4]:
# Create three tables - Artists, Museums, Paintings
try:
    
    # Use the cursor to interact with the database
    with connection.cursor() as cur:

        # Drop the tables if they already exist so they can be recreated
        cur.execute("DROP TABLE IF EXISTS paintings;")
        cur.execute("DROP TABLE IF EXISTS museums;")
        cur.execute("DROP TABLE IF EXISTS artists;")

        # Create 'artists' table to store artist information
        cur.execute("""
            CREATE TABLE IF NOT EXISTS artists (
                artist_id INT AUTO_INCREMENT PRIMARY KEY, # Generate unique artist IDs
                artist_name VARCHAR(255) NOT NULL UNIQUE, # Artist name assumed to be unique
                artist_nationality VARCHAR(225),
                artist_style VARCHAR(225),
                artist_birth SMALLINT,
                artist_death SMALLINT
            ) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci; # Use utf8mb4 charset for foreign character support
        """)

        # Create 'museums' table to store museum information
        cur.execute("""
            CREATE TABLE IF NOT EXISTS museums (
                museum_id INT AUTO_INCREMENT PRIMARY KEY,
                museum_name VARCHAR(255) NOT NULL UNIQUE,
                museum_city VARCHAR(225),
                museum_country VARCHAR(225)
            ) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci;
        """)

        # Create 'paintings' table to store paintings information
        cur.execute("""
            CREATE TABLE IF NOT EXISTS paintings (
                painting_id INT AUTO_INCREMENT PRIMARY KEY,
                painting_name VARCHAR(255) NOT NULL UNIQUE,
                painting_style VARCHAR(225),
                painting_subject VARCHAR(225),
                artist_id INT, # Foreign keys referencing the artist and museum tables
                museum_id INT,
                FOREIGN KEY (artist_id) REFERENCES artists(artist_id) ON DELETE SET NULL, # Set the ids to null if the record is deleted
                FOREIGN KEY (museum_id) REFERENCES museums(museum_id) ON DELETE SET NULL
            ) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci;
        """)

        # Commit the changes to the database to save the created tables
        connection.commit()

finally:
    # Close the connection
    # connection.close()
    pass

3) SQL code for populating tables. [5 marks]

In [5]:
# your code here

# Extract all columns from the DataFrame and remove dplicate rows
artists_data = df[['artist', 'artist_nationality', 'artist_style', 'artist_birth', 'artist_death']].drop_duplicates().values.tolist()
museums_data = df[['museum', 'museum_city', 'museum_country']].drop_duplicates().values.tolist()
paintings_data = df[['painting', 'painting_style', 'painting_subject', 'artist', 'museum']].drop_duplicates().values.tolist()

try:
    with connection.cursor() as cur:

        # Initialise dictionary to store the artist name as the key along with their id
        artist_ids = {}
        print("Inserting artists")

        # Loop through all artist data rows
        for artist in artists_data:
            try:

                # Check if the artist already exists in the artists table
                cur.execute("""
                    SELECT artist_id 
                    FROM artists 
                    WHERE artist_name = %(artist_name)s
                """, {"artist_name": artist[0]})

                # Fetch the results of the query
                check_duplicate = cur.fetchone()

                if check_duplicate: 
                    # If the artist does already exist, get their id
                    artist_id = check_duplicate[0]
                else:
                    # If the artist does not exist, insert a new artist
                    cur.execute("""
                        INSERT INTO artists (artist_name, artist_nationality, artist_style, artist_birth, artist_death)
                        VALUES (%(artist_name)s, %(artist_nationality)s, %(artist_style)s, %(artist_birth)s, %(artist_death)s)
                    """, {"artist_name": artist[0], "artist_nationality": artist[1], "artist_style": artist[2], 
                          "artist_birth": artist[3], "artist_death": artist[4]})
                    
                    # Get the artist id of the newly inserted artist
                    artist_id = cur.lastrowid

                # Add the artist name and ID in the directionary
                artist_ids[artist[0]] = artist_id 

            except Exception as e:
                # Prit an error message if there is an issue inserting the artist
                print(f"Failed to insert: {artist} — Error: {e}")
                raise # Stop the execution
        
        # Repeat the same process for the museums data
        museum_ids = {}
        print("Inserting museums")

        for museum in museums_data:
            try:
                cur.execute("""
                    SELECT museum_id 
                    FROM museums 
                    WHERE museum_name = %(museum_name)s
                """, {"museum_name": museum[0]}) #must have , to avoid potential errors, must be a tuple

                check_duplicate = cur.fetchone()

                if check_duplicate: 
                    museum_id = check_duplicate[0]

                else:
                    cur.execute("""
                        INSERT INTO museums (museum_name, museum_city, museum_country)
                        VALUES (%(museum_name)s, %(museum_city)s, %(museum_country)s)
                    """, {"museum_name": museum[0], "museum_city": museum[1], "museum_country": museum[2]})
                    
                    museum_id = cur.lastrowid

                museum_ids[museum[0]] = museum_id 
            
            except Exception as e:
                print(f"Failed to insert: {museum} — Error: {e}")
                raise
        
        # List to store the painting data, used with the "executemany" command
        painting_inserts = []
        print("Inserting paintings")

        # Loop through each painting data
        for painting in paintings_data:
            
            # Get the artist and museum id from the dictionaries
            artist_id = artist_ids[painting[3]]
            museum_id = museum_ids[painting[4]]

            # Append the data to the painting inserts list
            painting_inserts.append((painting[0], painting[1], painting[2], artist_id, museum_id))
        
        # Insert all paintings in a single query to optimize performance
        cur.executemany("""
            INSERT IGNORE INTO paintings (painting_name, painting_style, painting_subject, artist_id, museum_id)
            VALUES (%s, %s, %s, %s, %s)
        """, painting_inserts)

        #  Commit changes to the database
        connection.commit()
        
finally:
    pass

# 23 seconds to execute

Inserting artists
Inserting museums
Inserting paintings



PART B: QUERYING [20 marks] 

1 - First 10 artists with longest life  [4 marks]

Implement a query that returns the top 10 artist with longest life span based on their birth year and year of death for the whole dataset. You should calculate the life span of each artist and sort them in in descending order. Your query should return the results in 3 columns containing the names, the art styles and the nationality of the top 10 artists using aggr_scores..


In [6]:
# your code here
try:
    with connection.cursor() as cur:
        cur.execute(""" 
            SELECT artist_name, artist_nationality, artist_style FROM artists
            WHERE artist_birth IS NOT NULL AND artist_death IS NOT NULL
            ORDER BY (artist_death - artist_birth) DESC LIMIT 10;
        """)

        results = cur.fetchall()
        for row in results:
            print(row)
finally:
    pass       

('Sir George Clausen', 'English', 'Realist')
('Kees Van Dongen', 'Dutch', 'Fauvist')
('Titian', 'Italian', 'High Renaissance')
('Asher Brown Durand', 'American', 'Hudson River School')
('Katsushika Hokusai', 'Japanese', 'Ukiyo-e')
('Frank Weston Benson', 'American', 'Impressionist')
('Buonarroti Michelangelo', 'Italian', 'High Renaissance')
('Christian Rohlfs', 'German', 'Expressionist')
('Thomas Moran', 'American', 'Hudson River School')
('Thomas Sully', 'American', 'Portraitist')


2 - Find all the artists with style “Baroque” [4 marks]

Implement a query that returns all the names of all paintings and their artists with style “Baroque”.  Your query should return three columns: painting, artist, and painting_style.


In [7]:
# your code here
try:
    with connection.cursor() as cur:
        cur.execute(""" 
            SELECT paintings.painting_name, artists.artist_name, paintings.painting_style FROM paintings
            JOIN artists ON paintings.artist_id = artists.artist_id
            WHERE artists.artist_style = "Baroque";
        """)

        # Fetch all results returned by the query
        results = cur.fetchall()

        # Loop through the results and print the artists details
        for row in results:
            print(row)
finally:
    pass     

('Still Life with Flowers and a Watch', 'Abraham Mignon', 'Baroque')
("Still Life with Fruit and a Beaker on a Cock's Foot", 'Abraham Mignon', 'Baroque')
('Still Life with Fruit and a Goldfinch', 'Abraham Mignon', 'Baroque')
('Still Life with Fruit and Oysters', 'Abraham Mignon', 'Baroque')
('Still Life with Fruit, Oysters, and a Porcelain Bowl', 'Abraham Mignon', 'Baroque')
('The Overturned Bouquet', 'Abraham Mignon', 'Baroque')
('Arabian Horses at Pasture', 'Albrecht Adam', 'Baroque')
('Count Halm on His Basedow Estate', 'Albrecht Adam', 'Baroque')
('Horses at the Porch', 'Albrecht Adam', 'Baroque')
('Napoleon Before the Burning City of Smolensk', 'Albrecht Adam', 'Baroque')
('Peasants Resting in the Field', 'Albrecht Adam', 'Baroque')
('Portrait Oberleutnant Theodor Von Klein', 'Albrecht Adam', 'Baroque')
('The Rescue of Count Munnich', 'Albrecht Adam', 'Baroque')
('The Stable Yard', 'Albrecht Adam', 'Baroque')
('A Distant View of Dordrecht, with a Milkmaid and Four Cows', 'Aelbert 

3 - Find all the museums in the USA and return their total number  [4 marks]

Implement a query that returns all the museums in the USA and calculates the number of museums in that country. Your query should return the names of the museums and only one value for the total number of museums in the USA.

In [8]:
# your code here
try:
    with connection.cursor() as cur:
        cur.execute(""" 
            SELECT museum_name FROM museums WHERE museum_country = "USA"
        """)
        results = cur.fetchall()
        for row in results:
            print(row)

        cur.execute("""
            SELECT COUNT(*) FROM museums WHERE museum_country = "USA";
        """)
        count = cur.fetchone()[0]
        print(f"Total Museums in the USA:  {count}")
finally:
    pass  

('Los Angeles County Museum of Art',)
('The Metropolitan Museum of Art',)
('National Gallery of Art',)
('The J. Paul Getty Museum',)
('Cleveland Museum Of Art',)
('Philadelphia Museum of Art',)
('The Barnes Foundation',)
('The Art Institute of Chicago',)
('The Museum of Modern Art',)
('Solomon R. Guggenheim Museum',)
('Toledo Museum of Art',)
('Nelson-Atkins Museum of Art',)
('Saint Louis Art Museum',)
('Fine Arts Museums of San Francisco Legion of Honor',)
('The Phillips Collection',)
('Indianapolis Museum of Art',)
('Smithsonian American Art Museum',)
('Museum of Fine Arts, Houston',)
('Museum of Fine Arts Boston',)
('Walters Art Museum',)
('Columbus Museum of Art',)
('Kimbell Art Museum',)
('Dallas Museum of Art',)
('Chrysler Museum of Art',)
('Norton Simon Museum',)
Total Museums in the USA:  25


4 - Find all the Dutch artists who were Portraitist, and their paintings are displayed in a museum in Amsterdam . [4 marks]

Implement a query that retrieves all the artists who have Dutch nationality and Portraitist style with paintings displayed in a museum in Amsterdam city. The returned table should have three columns: artist, painting name and museum.


In [9]:
# your code here
try:
    with connection.cursor() as cur:
        cur.execute(""" 
            SELECT artists.artist_name, paintings.painting_name, museums.museum_name FROM artists
            JOIN paintings ON artists.artist_id = paintings.artist_id
            JOIN museums ON museums.museum_id = paintings.museum_id  
            WHERE artists.artist_style = "Portraitist" AND artists.artist_nationality = "Dutch" AND museums.museum_city = "Amsterdam";
        """)

        results = cur.fetchall()
        for row in results:
            print(row)
finally:
    pass  

('Adriaan De Lelie', 'Barend Klijn Barendsz, Poet', 'Rijksmuseum')
('Adriaan De Lelie', 'Cornelis Sebille Roos, Amsterdam Art Dealer and Keeper of the Nationale Konst-Gallery in Huis ten Bosch, The Hague', 'Rijksmuseum')
('Adriaan De Lelie', 'General Daendels Taking Leave of Lieutenant-Colonel Krayenhoff', 'Rijksmuseum')
('Adriaan De Lelie', 'Jonkheer Gysbert Carel Rutger Reinier van Brienen van Ramerus, with his Wife and four of their Children', 'Rijksmuseum')
('Adriaan De Lelie', 'Morning Visit', 'Rijksmuseum')
('Adriaan De Lelie', 'Portrait of Albertus Henricus Wiese, Governor-General of the Dutch East Indies', 'Rijksmuseum')
('Adriaan De Lelie', 'Portrait of Gerrit Verdooren, Vice-admiral', 'Rijksmuseum')
('Adriaan De Lelie', 'Portrait of Hendrick de Hartog', 'Rijksmuseum')
('Adriaan De Lelie', 'Portrait of the Family of Adrianus Bonebakker with Dirk L. Bennewitz', 'Rijksmuseum')
('Adriaan De Lelie', 'Portrait of Wilhelmina Maria Haack, fourth Wife of Gerrit Verdooren', 'Rijksmuseu

5 - Find all the French Expressionist painters who painted flowers and count how many of those paintings are displayed in New York, USA  [4 marks]

Implement a query that retrieves all the painter with French nationality with Expressionist style whose painting subjects were flowers. Your returned table should contain 4 columns: painting, artist, museum, museum_city. Then calculate how many of those flower paintings by French Expressionists are displayed in New York museums. This query should return only one value.


In [10]:
# your code here
# your code here
try:
    with connection.cursor() as cur:
        cur.execute(""" 
            WITH valid_paintings AS (
                SELECT paintings.painting_name, artists.artist_name, museums.museum_name, museums.museum_city FROM paintings
                JOIN artists ON artists.artist_id = paintings.artist_id
                JOIN museums ON museums.museum_id = paintings.museum_id  
                WHERE artists.artist_style = 'Expressionist' AND artists.artist_nationality = 'French' AND paintings.painting_subject = 'flowers'
            
            )
            
            SELECT painting_name, artist_name, museum_name, museum_city FROM valid_paintings
                    
        """)
        results = cur.fetchall()
        for row in results:
            print(row)
        
        cur.execute("""
            SELECT COUNT(*)
            FROM paintings
            JOIN artists ON artists.artist_id = paintings.artist_id
            JOIN museums ON museums.museum_id = paintings.museum_id
            WHERE artists.artist_style = 'Expressionist' AND artists.artist_nationality = 'French' AND paintings.painting_subject = 'flowers'
                AND museum_city = 'New York';
        """)
        count = cur.fetchone()[0]
        print(f"Total French Expressionist painters that painted Flowers in New York: {count}")
finally:
    pass  

('Bouquet of Roses', 'Albert André', '\\N', '\\N')
('Dahlias', 'Albert André', '\\N', '\\N')
('Flower Vase and Fruits', 'Albert André', '\\N', '\\N')
('Roses in a Glass', 'Albert André', '\\N', '\\N')
('Still Life', 'Albert André', '\\N', '\\N')
('The Bouquet (View from the Artist’s Studio, Paris)', 'Albert André', '\\N', '\\N')
('Vase of Flower by the Window', 'Albert André', '\\N', '\\N')
('Vase of Flowers and Fruits on the Table', 'Albert André', '\\N', '\\N')
('White Vase with Flowers', 'Albert André', '\\N', '\\N')
('Bouquet of Roses and Fruit', 'Albert André', 'The Metropolitan Museum of Art', 'New York')
('Bouquet of Flowers', 'Chaïm Soutine', 'The Barnes Foundation', 'Philadelphia')
Total French Expressionist painters that painted Flowers in New York: 1
